# Using an MLP for classification

In [ ]:
def load_data(json_path):
  with open(json_path,"r") as fp:
    data = json.load(fp)

  #convert lists into numpy arrays
  inputs = np.array(data["mfcc"])
  targets = np.array(data["labels"])

  return inputs,targets

In [ ]:
X,y = load_data(json_path)

In [ ]:
X

array([[[-2.12685546e+02,  1.00718600e+02, -1.23222813e+01, ...,
         -3.59266860e+00, -1.15427554e+00, -3.71990146e+00],
        [-2.10629281e+02,  1.03621303e+02, -1.16983593e+01, ...,
         -5.39652393e+00,  2.20071144e+00, -3.11685376e+00],
        [-1.95409321e+02,  1.04360651e+02, -1.85502341e-01, ...,
         -1.08171604e+01, -5.88807127e-01, -5.28410592e+00],
        ...,
        [-1.24157705e+02,  6.37352227e+01,  2.11729392e+01, ...,
         -1.65676625e+01,  1.37965153e+01, -2.31835349e+01],
        [-1.10560174e+02,  6.07024768e+01,  1.09157347e+00, ...,
         -1.29631778e+01,  1.39237288e+01, -1.62484843e+01],
        [-1.14273381e+02,  6.70723188e+01, -1.14650597e+01, ...,
         -9.90297501e+00,  1.22151307e+01, -9.50313534e+00]],

       [[-1.47248482e+02,  8.85711779e+01, -1.53050513e+01, ...,
         -1.19021387e+01,  6.63293296e+00, -8.55171762e+00],
        [-1.51392527e+02,  9.47541037e+01, -2.00325920e+01, ...,
         -9.10243162e+00,  9.11345492e

In [ ]:
y

array([0, 0, 0, ..., 9, 9, 9])

Now that we have the X and y for our classification, we'll divide it into train and test using sklearn

In [ ]:
from sklearn.model_selection import train_test_split

X_train,X_test,y_train,y_test = train_test_split(X,y,test_size = 0.33)

In [ ]:
X_train.shape

(6697, 130, 13)

And we are done with creating the dataset.

let's build a model with keras

In [ ]:
from tensorflow.keras import Sequential,layers

model = Sequential()

In [ ]:
#input layer
model.add(layers.Flatten(input_shape = (X.shape[1],X.shape[2])))

#1st hidden layer
model.add(layers.Dense(512,activation="relu"))

#2nd hidden layer
model.add(layers.Dense(256,activation="relu"))

#3rd hidden layer
model.add(layers.Dense(64,activation="relu"))

model.add(layers.Dense(10,activation="softmax")) #10 categories

In [ ]:
#Let's build the network
from keras.optimizers import Adam

optimizer = Adam(learning_rate=0.0001)
model.compile(optimizer = optimizer,
              loss = "sparse_categorical_crossentropy",
              metrics = ["accuracy"])

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten (Flatten)            (None, 1690)              0         
_________________________________________________________________
dense (Dense)                (None, 512)               865792    
_________________________________________________________________
dense_1 (Dense)              (None, 256)               131328    
_________________________________________________________________
dense_2 (Dense)              (None, 64)                16448     
_________________________________________________________________
dense_3 (Dense)              (None, 10)                650       
Total params: 1,014,218
Trainable params: 1,014,218
Non-trainable params: 0
_________________________________________________________________


Let's train the network

In [ ]:
model.fit(X_train,y_train,
          validation_data=(X_test,y_test),
          epochs = 50,
          batch_size=32)

Epoch 1/50
210/210 [==============================] - 3s 12ms/step - loss: 8.4355 - accuracy: 0.2800 - val_loss: 2.3317 - val_accuracy: 0.2019
Epoch 2/50
210/210 [==============================] - 2s 11ms/step - loss: 2.0895 - accuracy: 0.2465 - val_loss: 2.0790 - val_accuracy: 0.2828
Epoch 3/50
210/210 [==============================] - 2s 11ms/step - loss: 1.8797 - accuracy: 0.3339 - val_loss: 1.9951 - val_accuracy: 0.3407
Epoch 4/50
210/210 [==============================] - 2s 11ms/step - loss: 1.8021 - accuracy: 0.3545 - val_loss: 1.9267 - val_accuracy: 0.3253
Epoch 5/50
210/210 [==============================] - 2s 11ms/step - loss: 1.6795 - accuracy: 0.3872 - val_loss: 1.9012 - val_accuracy: 0.3528
Epoch 6/50
210/210 [==============================] - 2s 11ms/step - loss: 1.5892 - accuracy: 0.4269 - val_loss: 1.9304 - val_accuracy: 0.3146
Epoch 7/50
210/210 [==============================] - 2s 11ms/step - loss: 1.5334 - accuracy: 0.4589 - val_loss: 1.8630 - val_accuracy: 0.4210

If we compare the train accuracy vs test accuracy, then we observe that the model is overfitting. So we'll do some regularization or dropout to fix that.